# Iterative research results

This notebooks loads dataframe with experiment-based information and extracts basic info about its results.

In [1]:
# Necessary imports
import sys
import warnings
import pandas as pd
import numpy as np

sys.path.append('../..')
from seismiqb.batchflow.research import Results

warnings.filterwarnings("ignore")

The next two cells simply load dataframe from disk and transform it into more plausible one.

In [2]:
res_name = 'Research_iterative'
results = Results(res_name)
len(results.df)

5

In [3]:
columns = [
    'coverages', 'window_rates', 'corrs', 'local_corrs',    
]
df = results.df[columns + ['cube_and_horizon', 'repetition']]

df['cube_name'] = df['cube_and_horizon'].apply(lambda item: item.split('+')[0])
df['horizon_name'] = df['cube_and_horizon'].apply(lambda item: item.split('+')[1])

dff = (df
       .reset_index()
       .set_index(['cube_name', 'horizon_name'])[columns])

In [4]:
pd.set_option('display.max_rows', 10)

print(len(dff))
dff.head(10)

5


coverages  \
cube_name horizon_name                                                      
M_cube    t0_J1_anon    [0.9988092841813403, 0.9988092841813403, 0.999...   
          t0_B_anon     [0.996546924125887, 0.996546924125887, 0.99787...   
          t0_M_anon     [0.9969693447377448, 0.9969693447377448, 0.999...   
          t0_G_anon     [0.9979587728822977, 0.9979587728822977, 0.999...   
          t0_BV8_anon   [0.9675189734895628, 0.9675189734895628, 0.987...   

                                                             window_rates  \
cube_name horizon_name                                                      
M_cube    t0_J1_anon    [0.9977560460498376, 0.9977560460498376, 0.999...   
          t0_B_anon                                       [1.0, 1.0, 1.0]   
          t0_M_anon     [0.9999885078606233, 0.9999885078606233, 0.999...   
          t0_G_anon     [0.9997503851814872, 0.9997503851814872, 0.999...   
          t0_BV8_anon   [0.9680577939635701, 0.9680577939635701, 0.970...   

                                                                    corrs  \
cube_name horizon_name                                                      
M_cube    t0_J1_anon    [0.9042262554177692, 0.9042262554177692, 0.904...   
          t0_B_anon     [0.9645355741286292, 0.9645355741286292, 0.963...   
          t0_M_anon     [0.835260571264512, 0.835260571264512, 0.84253...   
          t0_G_anon     [0.840445385306522, 0.840445385306522, 0.84648...   
          t0_BV8_anon   [0.46532628394605663, 0.46532628394605663, 0.4...   

                                                              local_corrs  
cube_name horizon_name                                                     
M_cube    t0_J1_anon    [0.9818937404859199, 0.9818937404859199, 0.983...  
          t0_B_anon     [0.9865410404239828, 0.9865410404239828, 0.986...  
          t0_M_anon     [0.973125155215927, 0.973125155215927, 0.97309...  
          t0_G_anon     [0.9694810525563998, 0.9694810525563998, 0.969...  
          t0_BV8_anon   [0.9315018178347402, 0.9315018178347402, 0.934...

These are `coverage`s and metrics for every step of iterative research process:
- the first one is detection from a sparce carcass itself (column `0`)
- every odd number (1 and 3 in this case) are extension steps: in most cases, we can see the increase in the  amount of labeled points
- every even number (2 and 4 in this case) are enhancement steps: there is a slight improvement in `local_corrs` metric on these ones

In [5]:
wr = dff['window_rates'].apply(pd.Series).rename({i : f'window_rate_{i}' for i in range(3)}, axis=1)
cov = dff['coverages'].apply(pd.Series).rename({i : f'coverage_{i}' for i in range(3)}, axis=1)
corrs = dff['corrs'].apply(pd.Series).rename({i : f'corr_{i}' for i in range(3)}, axis=1)

pd.concat((wr, cov, corrs), axis=1).sort_index()

window_rate_0  window_rate_1  window_rate_2  \
cube_name horizon_name                                                
M_cube    t0_BV8_anon        0.968058       0.968058       0.970166   
          t0_B_anon          1.000000       1.000000       1.000000   
          t0_G_anon          0.999750       0.999750       0.999745   
          t0_J1_anon         0.997756       0.997756       0.999859   
          t0_M_anon          0.999989       0.999989       0.999997   

                        coverage_0  coverage_1  coverage_2    corr_0  \
cube_name horizon_name                                                 
M_cube    t0_BV8_anon     0.967519    0.967519    0.987670  0.465326   
          t0_B_anon       0.996547    0.996547    0.997871  0.964536   
          t0_G_anon       0.997959    0.997959    0.999586  0.840445   
          t0_J1_anon      0.998809    0.998809    0.999212  0.904226   
          t0_M_anon       0.996969    0.996969    0.999736  0.835261   

                          corr_1    corr_2  
cube_name horizon_name                      
M_cube    t0_BV8_anon   0.465326  0.486649  
          t0_B_anon     0.964536  0.963867  
          t0_G_anon     0.840445  0.846482  
          t0_J1_anon    0.904226  0.904998  
          t0_M_anon     0.835261  0.842532

Those three tables confirm that our postprocessing steps, namely extension and enhancement, do indeed enlarge the horizon and improve its quality. At the same time, the enhancement steps is not as noticeable and must be further developed.